In [ ]:
import csv
import json
from collections import defaultdict
import sys

if sys.version_info >= (3, 8):
    from typing import TypedDict  # pylint: disable=no-name-in-module
else:
    from typing_extensions import TypedDict

In [ ]:
file = "../../data/navigo_all_flows_1789.csv"
cargo_fields = ["commodity_purpose", "commodity_purpose2", "commodity_purpose3", "commodity_purpose4"]

In [ ]:
travels_from_dunkerque = []
destinations_states = set()
products = set()

with open(file, "r") as f:
    reader = csv.DictReader(f)
    for flow in reader:
        if flow["departure"] == "Dunkerque" and flow['departure_function'] == 'O':
            destinations_states.add(flow["destination_state_1789_fr"])
            for field in ["commodity_standardized_fr", "commodity_standardized2_fr", "commodity_standardized3_fr", "commodity_standardized4_fr"]:
                products.add(flow[field])
            travels_from_dunkerque.append(flow)
print(str(len(travels_from_dunkerque)) + " travels from dunkerque")
print('\n')
print("destination states :")
print('\n'.join(list(destinations_states)))
print('\n')
print('products :')
print('\n'.join(sorted(list(products))))

In [ ]:
import pandas as pd
# overview of cargos
number_of_commodity_purposes = {}
for flow in travels_from_dunkerque: 
    number_of_cargos = 0
    tonnage = float(flow["tonnage"]) if flow["tonnage"] != "" else 0.0
    for field in cargo_fields:
        if flow[field] != "":
            number_of_cargos = number_of_cargos + 1
    if number_of_cargos in number_of_commodity_purposes:
        number_of_commodity_purposes[number_of_cargos]["number of flows"] += 1        
        number_of_commodity_purposes[number_of_cargos]["tonnage"] += tonnage
    else:
        number_of_commodity_purposes[number_of_cargos] = {
            "number of flows": 1,
            "tonnage": tonnage
        }

print('Verification of the number of distinct cargos by trip:')
pd.DataFrame([{"number of cargos": nb, **detail} for (nb, detail) in number_of_commodity_purposes.items()])

In [ ]:
by_product_and_country = {}
for flow in travels_from_dunkerque:
    for field in cargo_fields:
        if flow[field] != '':
            product = flow[field]
            tonnage = float(flow['tonnage']) if flow['tonnage'] != '' else 0.0
            state = flow['destination_state_1789_fr']
            if product not in by_product_and_country:
                by_product_and_country[product] = {}
            if state not in by_product_and_country[product]:
                by_product_and_country[product][state] = {
                    "nb_flows": 0,
                    "tonnage": 0
                }
            by_product_and_country[product][state]['nb_flows'] += 1            
            by_product_and_country[product][state]['tonnage'] += tonnage

by_product_and_country_flat = []
for (product, rest) in by_product_and_country.items():
    for (state, metrics) in rest.items():
        by_product_and_country_flat.append({
            "product": product,
            "state": state,
            **metrics
        })

In [ ]:
from IPython.display import display
import pandas as pd

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
STOP_STATES = ['', 'Grande-Bretagne']
STOP_PRODUCTS = ['Lège', 'Etc.']

VegaLite({
  "title": "Valeurs par produit et partenaire pour les trajets au départ de Dunkerque",
  "data": {
      "values": [f for f in by_product_and_country_flat if f['state'] not in STOP_STATES and f['product'] not in STOP_PRODUCTS and 'pêche' not in f['product'].lower()]
         },
  "mark": {"type":"bar", "tooltip": {"content": "data"}},
  "encoding": {
    "x": {"field": "tonnage", "type": "quantitative", "title": "tonnage cumulé"},
    "y": {"field": "product", "sort": "-x", "title": "Produits"},
    "color": {
        "field": "state",
        "type": "nominal"
    }
  }
})

In [ ]:
# convert commodity purposes in toflit18
from dunkerquesprint import get_pointcalls_commodity_purposes_as_toflit_product

with_toflit18_products = get_pointcalls_commodity_purposes_as_toflit_product(from_dunkerque)